In [1]:
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../"))
sys.path.append(project_root)

In [2]:
def print_results(tuner):
    print("Final prompt:", tuner.final_prompt)
    print("Start prompt metric: ", tuner.init_metric)
    print("Final prompt metric: ", tuner.final_metric)

# Презентация PromptTuner - автопромптинг ассистента

Точка входа - ассистент. Он оптимизирует стартовый промпт.

Производится оптимизация промпта под переданный датасет (разбивается на train и validation), после чего благодаря заранее заданным мета-шаблонам вычислются метрики промптов на исходном датасете. Мета-шаблон можно посмотреть, вызвав соответствующий геттер ассистента.

In [3]:
from coolprompt.assistant import PromptTuner
tuner = PromptTuner()

WARNING 07-01 20:03:14 config.py:1865] Casting torch.bfloat16 to torch.float16.
INFO 07-01 20:03:23 config.py:350] This model supports multiple tasks: {'embedding', 'generate'}. Defaulting to 'generate'.
INFO 07-01 20:03:23 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='t-tech/T-lite-it-1.0', speculative_config=None, tokenizer='t-tech/T-lite-it-1.0', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execut

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 07-01 20:03:31 model_runner.py:1077] Loading model weights took 14.2426 GB
INFO 07-01 20:03:37 worker.py:232] Memory profiling results: total_gpu_memory=31.74GiB initial_memory_usage=14.66GiB peak_torch_memory=18.59GiB memory_usage_post_profile=14.69GiB non_torch_memory=0.44GiB kv_cache_size=9.53GiB gpu_memory_utilization=0.90
INFO 07-01 20:03:38 gpu_executor.py:113] # GPU blocks: 11154, # CPU blocks: 4681
INFO 07-01 20:03:38 gpu_executor.py:117] Maximum concurrency for 32768 tokens per request: 5.45x
INFO 07-01 20:03:42 model_runner.py:1400] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 07-01 20:03:42 model_runner.py:1404] If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilization` or switching to eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
IN

In [ ]:
print(tuner.get_task_prompt_template(task='classification', method='hype'))

{PROMPT}

### HARD CONSTRAINTS ###
This is an automated evaluation. Your output will be parsed by a script. Any deviation from the required format will result in failure.

1. OUTPUT FORMAT:
   - Output ONLY the final answer in the format: `<ans>LABEL</ans>`
   - LABEL MUST be EXACTLY one item from the list: [{LABELS}]
    - DO NOT include any explanation, reasoning, or extra text.
    - DO NOT include any meta-level commentary (e.g., "Sure", "Here is your answer", "Let's tackle this", "To answer this question", etc).
    - DO NOT modify the tag or the label format.
    - DO NOT repeat the answer or output multiple <ans> tags.

2. STOP CONDITION:
    - IMMEDIATELY stop generating after `</ans>`.
    - DO NOT output anything after `</ans>`.

3. FAILURE CONDITION:
    - If you break any of the above constraints, the output will be considered INVALID and REJECTED.

4. FORMAT EXAMPLE:
    1. Labels are [(A), (B), (C)] and you chose first answer  
       Output will be: <ans>(A)</ans>
    2.

In [ ]:
print(tuner.get_task_prompt_template(task='generation', method='distill'))

{PROMPT}

Generate the final answer bracketed with <ans> and </ans>.

INPUT:
{INPUT}

RESPONSE:



In [11]:
# Запуск метода hype без датасета
tuner.run(start_prompt="Write an essay about autumn")

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.43s/it, est. speed input: 175.14 toks/s, output: 39.93 toks/s]


'Write an essay about autumn, focusing on the changes in nature, the emotions it evokes, and the cultural significance of the season. Use descriptive language and provide examples from literature, art, or personal experiences to support your points.'

Ассистент в режиме оптимизации под датасет может решать одну из двух задач: классификацию или генерацию

In [4]:
from datasets import load_dataset

# Классификация
sst2 = load_dataset("stanfordnlp/sst2")
class_dataset = sst2['train']['sentence'][:100]
class_targets = sst2['train']['label'][:100]

Using custom data configuration stanfordnlp--sst2-c614fb49d6bf6d65
Reusing dataset parquet (/home/sitkina-alena/.cache/huggingface/datasets/stanfordnlp___parquet/stanfordnlp--sst2-c614fb49d6bf6d65/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
tuner.run(
    start_prompt="Classify sentence sentiment",
    task="classification",
    dataset=class_dataset,
    target=class_targets,
    metric="f1"
)

print_results(tuner)

Using custom data configuration stanfordnlp--sst2-c614fb49d6bf6d65
Reusing dataset parquet (/home/sitkina-alena/.cache/huggingface/datasets/stanfordnlp___parquet/stanfordnlp--sst2-c614fb49d6bf6d65/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


  0%|          | 0/3 [00:00<?, ?it/s]

Using the latest cached version of the module from /home/sitkina-alena/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--f1/34c46321f42186df33a6260966e34a368f14868d9cc2ba47d142112e2800d233 (last modified on Mon Jun 30 17:15:59 2025) since it couldn't be found locally at evaluate-metric--f1, or remotely on the Hugging Face Hub.
Processed prompts: 100%|██████████| 100/100 [00:06<00:00, 16.65it/s, est. speed input: 5170.06 toks/s, output: 133.22 toks/s]

Final prompt: Please provide a sentence and classify its sentiment as positive, negative, or neutral.
Start prompt metric:  0.6364983164983165
Final prompt metric:  0.8899889988998899


In [ ]:
# Генерация
samsum = load_dataset("knkarthick/samsum")
gen_dataset = samsum['train']['dialogue'][:200]
gen_targets = samsum['train']['summary'][:200]

tuner.run(
    start_prompt="Summarize the text",
    task="generation",
    dataset=gen_dataset,
    target=gen_targets,
    metric="meteor"
)

print_results(tuner)

[nltk_data] Downloading package wordnet to /home/sitkina-
[nltk_data]     alena/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/sitkina-
[nltk_data]     alena/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/sitkina-
[nltk_data]     alena/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
Processed prompts: 100%|██████████| 200/200 [00:25<00:00,  7.90it/s, est. speed input: 3109.76 toks/s, output: 255.11 toks/s]


Final prompt: Please provide a concise summary of the text, focusing on the main points and key details.
Start prompt metric:  0.25465833610131716
Final prompt metric:  0.3145034036877276


## Language model

При инициализации юзер может подать уже имеющуюся LLM-ку, 

или инициализировать асситента без аргументов, и будет использована дефолтная модель

In [ ]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

tuner_with_custom_llm = PromptTuner(model=llm)
tuner_with_custom_llm.run(start_prompt="Write an essay about autumn")

'Write a descriptive paragraph about the colors of autumn, focusing on the vibrant hues of red, orange, and yellow that characterize this season. Include sensory details to help the reader imagine the scene.'

Доступные модели и их провайдеры: https://python.langchain.com/docs/integrations/llms/

## Оптимизатор

Фреймворк поддерживает несколько различных алгоритмов оптимизации промптов

- HyPE
- DistillPrompt
- ReflectivePrompt

Выбор алгоритма производится в параметрах метода run

### HyPE
Мы уже видели его работу до этого - используя заранее заданную системную инструкцию просит модель оптимизировать поданный промпт. оптимизация происходит в одну итерацию

In [ ]:
tuner.run(
    start_prompt="Classify sentence sentiment",
    task="classification",
    dataset=class_dataset,
    target=class_targets,
)

print_results(tuner)

Processed prompts: 100%|██████████| 100/100 [00:05<00:00, 16.96it/s, est. speed input: 5265.82 toks/s, output: 135.69 toks/s]

Final prompt: Please provide a sentence and classify its sentiment as positive, negative, or neutral.
Start prompt metric:  0.6364983164983165
Final prompt metric:  0.8899889988998899


### DistillPrompt

Метод автопромптинга, основанный на последовательной дистилляции и агрегации знаний LLM

In [5]:
tuner.run(
    start_prompt="Classify sentence sentiment",
    task='classification',
    dataset=class_dataset[:20],
    target=class_targets[:20],
    method='distill',
    use_cache=False,
    num_epochs=1
)

print_results(tuner)

Using the latest cached version of the module from /home/sitkina-alena/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--f1/34c46321f42186df33a6260966e34a368f14868d9cc2ba47d142112e2800d233 (last modified on Mon Jun 30 17:15:59 2025) since it couldn't be found locally at evaluate-metric--f1, or remotely on the Hugging Face Hub.
[2025-07-01 20:04:49,726] - Starting DistillPrompt optimization...
Processed prompts: 100%|██████████| 15/15 [00:06<00:00,  2.42it/s, est. speed input: 469.53 toks/s, output: 303.43 toks/s]
[2025-07-01 20:29:25,224] - Best candidate score in round 0: 0.9321266968325792
[2025-07-01 20:29:25,225] - Best candidate prompt: Classify the sentiment of a given sentence as positive, negative, or neutral.
Processed prompts: 100%|██████████| 5/5 [00:00<00:00, 17.36it/s, est. speed input: 1010.82 toks/s, output: 138.93 toks/s]
[2025-07-01 20:29:25,530] - Final best prompt score on validation: 0.7619047619047619
[2025-07-01 20:29:25,531] - Final best prompt

Final prompt: Classify the sentiment of a given sentence as positive, negative, or neutral.
Start prompt metric:  0.21031746031746032
Final prompt metric:  0.8465473145780051


### ReflectivePrompt

Метод автопромптинга на основе эволюционных алгоритмов, использующий подход рефлексивной эволюции для более точного и расширенного поиска оптимальных промптов

In [6]:
tuner.run(
    start_prompt="Perform Sentiment Classification task.",
    task='classification',
    dataset=class_dataset[:20],
    target=class_targets[:20],
    method='reflective',
    problem_description='sentiment classification',
    use_cache=False,
    population_size=4,
    num_epochs=1
)

print_results(tuner)

Using the latest cached version of the module from /home/sitkina-alena/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--f1/34c46321f42186df33a6260966e34a368f14868d9cc2ba47d142112e2800d233 (last modified on Mon Jun 30 17:15:59 2025) since it couldn't be found locally at evaluate-metric--f1, or remotely on the Hugging Face Hub.
[2025-07-01 20:32:32,142] - Initializing population...
Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.63s/it, est. speed input: 49.23 toks/s, output: 39.99 toks/s]
[2025-07-01 20:32:33,773] - Evaluating population...
Processed prompts: 100%|██████████| 4/4 [00:01<00:00,  2.36it/s, est. speed input: 250.48 toks/s, output: 130.84 toks/s]
[2025-07-01 20:41:59,395] - Evaluating population...
Processed prompts: 100%|██████████| 15/15 [01:46<00:00,  7.07s/it, est. speed input: 9.10 toks/s, output: 114.16 toks/s]   
[2025-07-01 20:43:48,079] - Iteration 0
                Elitist (0.9282296650717703):
                Carry out a Sentiment Ana

Final prompt: Carry out a Sentiment Analysis task to categorize text sentiments.
Start prompt metric:  0.4373401534526854
Final prompt metric:  0.9466666666666667


## Evaluator

Если вы укажете датасет для оптимизации, фреймворк позволяет измерять метрики, получаемые при использовании определенного промпта

На данный момент поддерживаются
- accuracy и f1 для классификации
- meteor, rouge и bleu для генерации

Метрики находятся в публичных полях ассистента

In [7]:
print("Start prompt metric: ", tuner.init_metric)
print("Final prompt metric: ", tuner.final_metric)

Start prompt metric:  0.4373401534526854
Final prompt metric:  0.9466666666666667
